# Deduplicação de Publicações Acadêmicas

Este notebook realiza a deduplicação de publicações acadêmicas dos professores, criando o arquivo `publicacoes_unificadas.json` na pasta `data`.

O processo inclui:
1. Carregamento dos dados dos professores
2. Normalização de nomes e títulos
3. Agrupamento de publicações similares
4. Identificação de colaborações
5. Geração do arquivo JSON unificado

In [1]:
import json
import re
import unicodedata
import uuid
from collections import defaultdict

In [2]:
# Carregar dados dos professores
with open("../data/professores.json", encoding="utf-8") as f:
    professores = json.load(f)

print(f"Total de professores carregados: {len(professores)}")

Total de professores carregados: 67


In [3]:
def normalize_name_for_comparison(name):
    """Normaliza nomes para comparação, removendo acentos e caracteres especiais."""
    if not name or not isinstance(name, str):
        return ""

    # Remove acentos
    name = unicodedata.normalize("NFD", name)
    name = "".join(char for char in name if unicodedata.category(char) != "Mn")

    # Converte para maiúsculo e remove pontuação
    name = name.upper()
    name = re.sub(r"[^\w\s]", " ", name)
    name = " ".join(name.split())
    return name


def extract_name_variations(name):
    """Extrai variações de um nome para matching."""
    if not name:
        return set()

    normalized = normalize_name_for_comparison(name)
    variations = {normalized}
    variations.add(normalized.replace(" ", ""))

    parts = normalized.split()
    if len(parts) > 1:
        # Iniciais
        initials = "".join([part[0] for part in parts if part])
        variations.add(initials)

    # Primeiro e último nome
    if len(parts) > 2:
        first_last = f"{parts[0]} {parts[-1]}"
        variations.add(first_last)
        variations.add(first_last.replace(" ", ""))

    return variations


def create_comprehensive_name_mapping(professors_data):
    """Cria mapeamento abrangente de nomes para identificação de autores."""
    name_mapping = {}
    professor_info = {}

    for prof in professors_data:
        identificacao = prof.get("identificacao", {})
        nome_principal = identificacao.get("nome", "")
        lattes_id = identificacao.get("lattes_id", "")
        nomes_citacao = identificacao.get("nomes_citacao", [])

        if not nome_principal:
            continue

        professor_info[lattes_id] = {
            "nome_principal": nome_principal,
            "nomes_citacao": nomes_citacao,
        }

        # Mapear nome principal
        normalized_main = normalize_name_for_comparison(nome_principal)
        name_mapping[normalized_main] = {
            "nome_principal": nome_principal,
            "lattes_id": lattes_id,
        }

        # Mapear variações do nome principal
        for variation in extract_name_variations(nome_principal):
            name_mapping[variation] = {
                "nome_principal": nome_principal,
                "lattes_id": lattes_id,
            }

        # Mapear nomes de citação
        for nome_citacao in nomes_citacao:
            if nome_citacao and isinstance(nome_citacao, str):
                normalized_citation = normalize_name_for_comparison(nome_citacao)
                name_mapping[normalized_citation] = {
                    "nome_principal": nome_principal,
                    "lattes_id": lattes_id,
                }

                # Mapear variações dos nomes de citação
                for variation in extract_name_variations(nome_citacao):
                    name_mapping[variation] = {
                        "nome_principal": nome_principal,
                        "lattes_id": lattes_id,
                    }

    return name_mapping, professor_info

In [4]:
def advanced_author_matching(author_name, name_mapping):
    """Realiza matching avançado de autores usando o mapeamento de nomes."""
    if not author_name or not isinstance(author_name, str):
        return None

    normalized_author = normalize_name_for_comparison(author_name)

    # Busca direta
    if normalized_author in name_mapping:
        return name_mapping[normalized_author]

    # Busca sem espaços
    no_spaces = normalized_author.replace(" ", "")
    if no_spaces in name_mapping:
        return name_mapping[no_spaces]

    # Busca por primeiro e último nome
    author_parts = normalized_author.split()
    if len(author_parts) >= 2:
        first_last = f"{author_parts[0]} {author_parts[-1]}"
        if first_last in name_mapping:
            return name_mapping[first_last]

        first_last_no_space = first_last.replace(" ", "")
        if first_last_no_space in name_mapping:
            return name_mapping[first_last_no_space]

    # Busca por partes significativas comuns
    for mapped_name, prof_info in name_mapping.items():
        mapped_parts = set(mapped_name.split())
        author_parts_set = set(author_parts)

        common_parts = mapped_parts.intersection(author_parts_set)
        if len(common_parts) >= 2:
            # Verificar se há partes significativas (> 3 caracteres)
            significant_common = [part for part in common_parts if len(part) > 3]
            if significant_common:
                return prof_info

    return None

In [5]:
def normalize_text(text):
    """Normaliza texto removendo acentos e caracteres especiais."""
    if not text:
        return ""

    text = unicodedata.normalize("NFD", text)
    text = "".join(char for char in text if unicodedata.category(char) != "Mn")
    text = text.lower()
    text = re.sub(r"[^\w\s]", " ", text)
    text = " ".join(text.split())
    return text


def extract_main_title(full_title):
    """Extrai o título principal de um título completo."""
    if not full_title:
        return ""

    # Divide por pontos seguidos de espaço
    parts = re.split(r"\.\s+", full_title)

    # Procura por indicadores de informações bibliográficas
    for i, part in enumerate(parts):
        if re.search(
            r"\b(v\.|vol\.|volume|p\.|pp\.|páginas|anais|proceedings|revista)",
            part,
            re.IGNORECASE,
        ):
            return ". ".join(parts[:i]).strip()
        if re.search(r"\b\d{4}\s*\.$", part):
            return ". ".join(parts[: i + 1]).strip()

    # Se a primeira parte é significativa, use-a
    if len(parts) > 1 and len(parts[0].strip()) > 20:
        return parts[0].strip()

    return full_title.strip()

In [6]:
def extract_all_publications(professors_data):
    """Extrai todas as publicações dos professores."""
    publications = []

    for professor in professors_data:
        prof_name = professor.get("identificacao", {}).get("nome", "Desconhecido")
        prof_id = professor.get("identificacao", {}).get("lattes_id", "")

        for pub in professor.get("producao_bibliografica", []):
            titulo_completo = pub.get("titulo", "").strip()
            if titulo_completo:
                titulo_principal = extract_main_title(titulo_completo)
                publications.append(
                    {
                        "professor": prof_name,
                        "professor_id": prof_id,
                        "titulo_completo": titulo_completo,
                        "titulo_principal": titulo_principal,
                        "titulo_normalizado": normalize_text(titulo_principal),
                        "autores": pub.get("autores", []),
                        "ano": pub.get("ano", ""),
                        "revista": pub.get("revista"),
                        "doi": pub.get("doi"),
                        "paginas": pub.get("paginas"),
                    }
                )

    return publications


def group_publications_by_similarity(publications):
    """Agrupa publicações por similaridade de título."""
    title_groups = defaultdict(list)
    for pub in publications:
        title_groups[pub["titulo_normalizado"]].append(pub)
    return list(title_groups.values())

In [7]:
def deduplicate_with_enhanced_names(professores, name_mapping):
    """Realiza deduplicação avançada com mapeamento de nomes melhorado."""
    publicacoes = extract_all_publications(professores)
    grupos = group_publications_by_similarity(publicacoes)
    publicacoes_unicas = []
    total_colaboracoes = 0

    for grupo in grupos:
        # Escolhe a publicação com mais autores como representativa
        def count_authors(pub):
            autores = pub.get("autores", "")
            if isinstance(autores, str):
                return len(autores.split(";"))
            elif isinstance(autores, list):
                return len(autores)
            else:
                return 0

        publicacao_repr = max(grupo, key=count_authors)
        unique_id = str(uuid.uuid4())
        professores_associados = set()
        nomes_autores_originais = set()

        # Processar cada publicação do grupo
        for pub in grupo:
            if "professor" in pub:
                professores_associados.add(pub["professor"])

            autores = pub.get("autores", "")
            if isinstance(autores, str):
                autores_list = autores.split(";")
            elif isinstance(autores, list):
                autores_list = autores
            else:
                autores_list = []

            # Processar cada autor
            for autor in autores_list:
                autor = str(autor).strip()
                if autor:
                    nomes_autores_originais.add(autor)
                    match = advanced_author_matching(autor, name_mapping)
                    if match:
                        professores_associados.add(match["nome_principal"])

        eh_colaboracao = len(professores_associados) > 1
        if eh_colaboracao:
            total_colaboracoes += 1

        publicacao_unica = {
            "id_unico": unique_id,
            "titulo": publicacao_repr.get("titulo", ""),
            "ano": publicacao_repr.get("ano", ""),
            "autores_originais": list(nomes_autores_originais),
            "professores_identificados": list(professores_associados),
            "eh_colaboracao": eh_colaboracao,
            "num_professores": len(professores_associados),
            "publicacoes_originais": len(grupo),
            "detalhes_originais": grupo,
        }

        publicacoes_unicas.append(publicacao_unica)

    return publicacoes_unicas

In [8]:
def create_simplified_json(publicacoes_unicas):
    """Cria versão simplificada do JSON para o arquivo final."""
    publicacoes_simplificadas = []

    for pub in publicacoes_unicas:
        titulo = ""
        if "detalhes_originais" in pub and pub["detalhes_originais"]:
            original = pub["detalhes_originais"][0]
            titulo = (
                original.get("titulo_completo")
                or original.get("titulo_principal")
                or original.get("titulo")
                or ""
            )

        publicacao_simples = {
            "id_unico": pub["id_unico"],
            "titulo": titulo,
            "professores": pub["professores_identificados"],
        }
        publicacoes_simplificadas.append(publicacao_simples)

    return publicacoes_simplificadas

In [9]:
# Executar o processo de deduplicação
print("Criando mapeamento de nomes...")
name_mapping, professor_info = create_comprehensive_name_mapping(professores)
print(f"Mapeamento criado com {len(name_mapping)} entradas")

print("\nRealizando deduplicação...")
publicacoes_unicas = deduplicate_with_enhanced_names(professores, name_mapping)
print(f"Encontradas {len(publicacoes_unicas)} publicações únicas")

# Contar colaborações
colaboracoes = [pub for pub in publicacoes_unicas if pub["eh_colaboracao"]]
print(f"Colaborações identificadas: {len(colaboracoes)}")
print(f"Publicações individuais: {len(publicacoes_unicas) - len(colaboracoes)}")

Criando mapeamento de nomes...
Mapeamento criado com 1670 entradas

Realizando deduplicação...
Encontradas 5002 publicações únicas
Colaborações identificadas: 1997
Publicações individuais: 3005


In [ ]:
# Criar arquivo JSON simplificado
print("Criando arquivo JSON simplificado...")
publicacoes_json_simples = create_simplified_json(publicacoes_unicas)

# Salvar o arquivo publicacoes_unificadas.json
output_file = "../data/publicacoes_unificadas.json"
with open(output_file, "w", encoding="utf-8") as f:
    json.dump(publicacoes_json_simples, f, ensure_ascii=False, indent=2)

print(f"Arquivo salvo em: {output_file}")
print(f"Total de publicações no arquivo: {len(publicacoes_json_simples)}")

# Exibir algumas estatísticas
print("\n" + "=" * 50)
print("ESTATÍSTICAS FINAIS")
print("=" * 50)
print(
    f"Total de publicações processadas: {sum(len(p.get('producao_bibliografica', [])) for p in professores)}"
)
print(f"Publicações únicas após deduplicação: {len(publicacoes_json_simples)}")
print(
    f"Taxa de deduplicação: {(1 - len(publicacoes_json_simples) / sum(len(p.get('producao_bibliografica', [])) for p in professores)) * 100:.1f}%"
)

# Exemplo de publicação
if publicacoes_json_simples:
    print("\nExemplo de publicação:")
    exemplo = publicacoes_json_simples[0]
    print(f"  ID: {exemplo.get('id_unico', 'N/A')}")
    print(f"  Título: {exemplo.get('titulo', 'N/A')[:100]}...")
    print(f"  Professores: {exemplo.get('professores', [])}")

Criando arquivo JSON simplificado...
Arquivo salvo em: ../data/publicacoes_unificadas.json
Total de publicações no arquivo: 5002

ESTATÍSTICAS FINAIS
Total de publicações processadas: 5382
Publicações únicas após deduplicação: 5002
Taxa de deduplicação: 7.1%

Exemplo de publicação:
  ID: abce666d-53be-44af-9f80-de06e258f282
  Título: Supporting the choice of the best-fit agile model using FITradeoff. PESQUISA OPERACIONAL (IMPRESSO) ...
  Professores: ['Adriana Carla Damasceno']


## Verificação do Arquivo Gerado

O arquivo `publicacoes_unificadas.json` foi criado na pasta `data` com a estrutura:

```json
[
  {
    "id_unico": "uuid-único",
    "titulo": "Título da publicação",
    "professores": ["Nome Professor 1", "Nome Professor 2", ...]
  },
  ...
]
```

Este arquivo contém:
- Publicações deduplicadas
- Identificação de colaborações entre professores
- IDs únicos para cada publicação
- Lista de professores envolvidos em cada publicação